In [1]:
import os
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import gc
import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
# df_mulliken_charges = pd.read_csv(FOLDER + 'mulliken_charges.csv')
# df_sample =  pd.read_csv(FOLDER + 'sample_submission.csv')
# df_magnetic_shielding_tensors = pd.read_csv(FOLDER + 'magnetic_shielding_tensors.csv')
# df_train = pd.read_csv(FOLDER + 'train.csv')
# df_test = pd.read_csv(FOLDER + 'test.csv')
# df_dipole_moments = pd.read_csv(FOLDER + 'dipole_moments.csv')
# df_potential_energy = pd.read_csv(FOLDER + 'potential_energy.csv')
# df_structures = pd.read_csv(FOLDER + 'structures.csv')
# df_scalar_coupling_contributions = pd.read_csv(FOLDER + 'scalar_coupling_contributions.csv')

In [4]:
try_test = '20190721_dist_ang_ori_bond_cos_test_{}.pickle'
trained_models = '20190721_champs_models_lgb_{}.pickle'
submission_name = '20190721_submission.csv'
submission_temp = 'submission_temp_{}.pickle'

In [5]:
bond_types = ['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN']

In [6]:
for i, b in enumerate(bond_types):
    print('-'*10, b, '-'*10)
    start = time.time()
    df_bond = pd.read_pickle(OUTPUT +try_test.format(b))
    y_pred_b =[]

    with open(OUTPUT + trained_models.format(b), 'rb') as f:
        models_b = pickle.load(f)

    X_test = df_bond[df_bond.columns[5:]].values  
    num_seed = len(models_b)
    for s in range(num_seed):
        gbm = models_b[s]
        y_pred_single = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        y_pred_b.append(y_pred_single)
        
    y_pred = np.array(y_pred_b).mean(axis=0)
    
    df_submit_b = pd.DataFrame([df_bond['id'], y_pred], index=['id', 'scalar_coupling_constant']).T
    df_submit_b.to_pickle(OUTPUT +submission_temp.format(b))
    
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    gc.collect()

---------- 1JHC ----------
elapsed_time:1197.4483001232147[sec]
---------- 2JHH ----------
elapsed_time:421.19561672210693[sec]
---------- 1JHN ----------
elapsed_time:14.156259536743164[sec]
---------- 2JHN ----------
elapsed_time:46.87821340560913[sec]
---------- 2JHC ----------
elapsed_time:1521.6596581935883[sec]
---------- 3JHH ----------
elapsed_time:893.0700449943542[sec]
---------- 3JHC ----------
elapsed_time:2036.7518043518066[sec]
---------- 3JHN ----------
elapsed_time:76.8850302696228[sec]


In [7]:
df_submit = pd.DataFrame()
for b in bond_types:
    df_submit_b = pd.read_pickle(OUTPUT +submission_temp.format(b))
    df_submit = pd.concat([df_submit, df_submit_b], axis=0)

In [8]:
df_submit['id'] = df_submit['id'].astype('int32')

In [9]:
df_submit.head()

,id,scalar_coupling_constant
0,4658148,196.898820
1,4658150,196.898820
2,4658152,90.810420
3,4658156,90.809981
4,4658159,82.820017


In [10]:
df_submit.to_csv(OUTPUT + submission_name, index=False)